In [1]:
import numpy as np
import pandas as pd
import os

## Import Data

In [2]:
# set the path of the processed data
processed_data_path = os.path.join(os.path.pardir, 'data', 'processed')
train_file_path = os.path.join(processed_data_path, 'train.csv')
test_file_path = os.path.join(processed_data_path, 'test.csv')

In [3]:
train_df = pd.read_csv(train_file_path, index_col='PassengerId')
test_df = pd.read_csv(test_file_path, index_col='PassengerId')

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 33 columns):
Survived              891 non-null int64
Age                   891 non-null float64
Fare                  891 non-null float64
FamilySize            891 non-null int64
IsMother              891 non-null int64
IsMale                891 non-null int64
Deck_A                891 non-null int64
Deck_B                891 non-null int64
Deck_C                891 non-null int64
Deck_D                891 non-null int64
Deck_E                891 non-null int64
Deck_F                891 non-null int64
Deck_G                891 non-null int64
Deck_Z                891 non-null int64
Pclass_1              891 non-null int64
Pclass_2              891 non-null int64
Pclass_3              891 non-null int64
Title_Lady            891 non-null int64
Title_Master          891 non-null int64
Title_Miss            891 non-null int64
Title_Mr              891 non-null int64
Title_Mrs             891 non-

In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 32 columns):
Age                   418 non-null float64
Fare                  418 non-null float64
FamilySize            418 non-null int64
IsMother              418 non-null int64
IsMale                418 non-null int64
Deck_A                418 non-null int64
Deck_B                418 non-null int64
Deck_C                418 non-null int64
Deck_D                418 non-null int64
Deck_E                418 non-null int64
Deck_F                418 non-null int64
Deck_G                418 non-null int64
Deck_Z                418 non-null int64
Pclass_1              418 non-null int64
Pclass_2              418 non-null int64
Pclass_3              418 non-null int64
Title_Lady            418 non-null int64
Title_Master          418 non-null int64
Title_Miss            418 non-null int64
Title_Mr              418 non-null int64
Title_Mrs             418 non-null int64
Title_Officer         418 n

## Data Prepartion

In [4]:
X = train_df.loc[:,'Age':].as_matrix().astype('float') # ML model needs data in numeric format.  
y = train_df['Survived'].ravel()         # ravel will return flattened 1-dimension array or vector

In [5]:
print X.shape, y.shape

(891L, 32L) (891L,)


In [6]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print X_train.shape, y_train.shape
print X_test.shape, y_test.shape

(712L, 32L) (712L,)
(179L, 32L) (179L,)


In [7]:
# average survival rate in train and test split
print('mean survival rate in train : {0:.3f}'.format(np.mean(y_train)))
print('mean survival rate in test : {0:.3f}'.format(np.mean(y_test)))

mean survival rate in train : 0.383
mean survival rate in test : 0.385


## Build and evaluate Baseline model

In [8]:
import sklearn

In [9]:
sklearn.__version__

'0.19.1'

## Baseline model

In [10]:
from sklearn.dummy import DummyClassifier

In [11]:
# create model
model_dummy = DummyClassifier(strategy='most_frequent', random_state=0)

In [12]:
# train model
model_dummy.fit(X_train, y_train)

DummyClassifier(constant=None, random_state=0, strategy='most_frequent')

In [13]:
# Accuracy metric
print('score for baseline model : {0:.2f}'.format(model_dummy.score(X_test, y_test)))

score for baseline model : 0.61


In [38]:
# Performance matrix
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [39]:
# accuracy score
print('accuracy for baseline model : {0:.2f}'.format(accuracy_score(y_test, model_dummy.predict(X_test))))

accuracy for baseline model : 0.61


In [40]:
# confusion matrix
print('confusion matrix for baseline model : \n {0}'.format(confusion_matrix(y_test, model_dummy.predict(X_test))))

confusion matrix for baseline model : 
 [[110   0]
 [ 69   0]]


In [41]:
# precision and recall score
print('precision for baseline model : {0:.2f}'.format(precision_score(y_test, model_dummy.predict(X_test))))
print('recall for baseline model : {0:.2f}'.format(recall_score(y_test, model_dummy.predict(X_test))))

precision for baseline model : 0.00
recall for baseline model : 0.00


## First Kaggle submission

In [42]:
# converting test df to matrix
test_X = test_df.as_matrix().astype('float')

In [43]:
# get predications
predictions = model_dummy.predict(test_X)

In [44]:
# create submission file with Passenger Id and Predicted survival
df_submission = pd.DataFrame({'PassengerId':test_df.index, 'Survived': predictions})

In [45]:
df_submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [46]:
submission_data_path = os.path.join(os.path.pardir, 'data', 'external')
submission_file_path = os.path.join(submission_data_path, '01_dummy.csv')

In [47]:
# write to file
df_submission.to_csv(submission_file_path, index=False)  # else index column would be added

In [48]:
# create utility method
def get_submission_file(model, filename):
    # converting to matrix
    test_X = test_df.as_matrix().astype('float')
    # get predications
    predictions = model.predict(test_X)
    # create submission file with Passenger Id and Predicted survival
    df_submission = pd.DataFrame({'PassengerId':test_df.index, 'Survived': predictions})
    # submission file
    submission_data_path = os.path.join(os.path.pardir, 'data', 'external')
    submission_file_path = os.path.join(submission_data_path, filename)
    # write to file
    df_submission.to_csv(submission_file_path, index=False)  # else index column would be added

In [49]:
# get submission file
get_submission_file(model_dummy,'01_dummy.csv')

## Logistic Regression Model

In [50]:
from sklearn.linear_model import LogisticRegression

In [51]:
# create model
model_lr_1 = LogisticRegression(random_state=0)

In [52]:
# train model
model_lr_1.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [53]:
# evaluate model
print('score for logestic regression model - version 1 : {0:.2f}'.format(model_lr_1.score(X_test, y_test)))

score for logestic regression model - version 1 : 0.83


In [54]:
# accuracy score
print('accuracy for logestic regression model - version 1 : {0:.2f}'.format(accuracy_score(y_test, model_lr_1.predict(X_test))))
# confusion matrix
print('confusion matrix for logestic regression model - version 1 : \n {0}'.format(confusion_matrix(y_test, model_lr_1.predict(X_test))))
# precision and recall score
print('precision for logestic regression model - version 1 : {0:.2f}'.format(precision_score(y_test, model_lr_1.predict(X_test))))
print('recall for logestic regression model - version 1 : {0:.2f}'.format(recall_score(y_test, model_lr_1.predict(X_test))))

accuracy for logestic regression model - version 1 : 0.83
confusion matrix for logestic regression model - version 1 : 
 [[95 15]
 [15 54]]
precision for logestic regression model - version 1 : 0.78
recall for logestic regression model - version 1 : 0.78


In [55]:
# model coefficents
model_lr_1.coef_

array([[-0.0284227 ,  0.00455451, -0.50009091,  0.61781314, -0.8139233 ,
         0.12845079, -0.1728179 , -0.39317833,  0.52159976,  1.09941224,
         0.40341217, -0.18345051, -0.30036041,  0.96533486,  0.48256743,
        -0.34483447,  0.28089591,  1.21761326,  0.56363966, -1.44586303,
         1.07245555, -0.11273706, -0.47293647,  0.16255645,  0.24716927,
         0.28009442,  0.41324768,  0.49183529,  0.46198829,  0.14924424,
         0.37283518,  0.73023264]])

In [56]:
# get submission file
get_submission_file(model_lr_1,'02_lr.csv')

## Part 2 - Hyperparameter Optimization

In [57]:
# Base model
model_lr = LogisticRegression(random_state=0)

In [58]:
from sklearn.model_selection import GridSearchCV

In [63]:
# Set Hyperparameter values
parameters = {'C':[1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty':['l1', 'l2']} # l1, l2
clf = GridSearchCV(model_lr, param_grid=parameters, cv=3)

In [64]:
# Fit the model
clf.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [1.0, 10.0, 50.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [66]:
# Get the best settings
clf.best_params_

{'C': 1.0, 'penalty': 'l1'}

In [67]:
print('best score : {0:.2f}'.format(clf.best_score_))

best score : 0.83


In [68]:
# evaluate model
print('score for Logistic Regression - version 2 : {0:.2f}'.format(clf.score(X_test, y_test)))

score for Logistic Regression - version 2 : 0.83


## Making Third Submission

In [69]:
# get submission file
get_submission_file(clf, '03_lr.csv')

## Feature Normalization and Standardization

In [70]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

## Feature Normalization

In [71]:
# normalize training data to same scale
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [73]:
# Check scaled values
X_train_scaled[:,0].min, X_train_scaled[:,0].max()

(<function min>, 1.0)

In [74]:
# normalize test data
X_test_scaled = scaler.transform(X_test)

## Feature Standization

In [75]:
# Feature standardization to 0 mean and unir variance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Create model after standardization

In [77]:
# base model
model_lr = LogisticRegression()
parameters = {'C':[1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty':['l1', 'l2']} # l1, l2
clf = GridSearchCV(model_lr, param_grid=parameters, cv=3)
clf.fit(X_train_scaled, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [1.0, 10.0, 50.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [79]:
clf.best_score_

0.8132022471910112

In [80]:
# evaluate model
print('score for Logistic Regression - version 2 : {0:.2f}'.format(clf.score(X_test_scaled, y_test)))

score for Logistic Regression - version 2 : 0.84


## Model Persistence

In [81]:
# import pickle library
import pickle

In [82]:
# create file paths
model_file_path = os.path.join(os.path.pardir, 'models', 'lr_model.pkl') # model file
scaler_file_path = os.path.join(os.path.pardir, 'models', 'lr_scaler.pkl') # scaler file

In [83]:
# open the files to write
model_file_pickle = open(model_file_path, 'wb')
scaler_file_pickle = open(scaler_file_path, 'wb')

In [84]:
# persist the model and scaler
pickle.dump(clf, model_file_pickle)
pickle.dump(scaler, scaler_file_pickle)

In [85]:
# close the file
model_file_pickle.close()
scaler_file_pickle.close()

## Load the persisted files

In [86]:
# open files in read mode
model_file_pickle = open(model_file_path, 'r')
scaler_file_pickle = open(scaler_file_path, 'r')
# load files
clf_loaded = pickle.load(model_file_pickle)
scaler_loaded = pickle.load(scaler_file_pickle)
# close files
model_file_pickle.close()
scaler_file_pickle.close()

In [87]:
clf_loaded

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [1.0, 10.0, 50.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [88]:
scaler_loaded

StandardScaler(copy=True, with_mean=True, with_std=True)

In [89]:
# transform the test data using the loaded scaler object
X_test_scaled = scaler_loaded.transform(X_test)
# calculate the score using loaded model 
print('score for persisted Logistic Regression : {0:.2f}'.format(clf.score(X_test_scaled, y_test)))

score for persisted Logistic Regression : 0.84
